In [1]:
from hamest import *

Using TensorFlow backend.


In [2]:
import numpy as np

In [3]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TerminateOnNaN

In [4]:
average_over = 1000
batches_worth_of_data = 1
batch_size = 512
max_epochs = 50
steps_per_epoch = batches_worth_of_data*2

model_checkpoint_filepath = 'm_checkpoint'
model1_checkpoint_filepath = 'm1_checkpoint'

In [5]:
join_hists = lambda h: {k:(sum((_.get(k,[None]) for _ in h),[])) for k in h[-1].keys()}

# Model dependent stuff

This is the source of training data. It is implemented in a separate module.

In [6]:
from ringham import *

n = 3
SPAM = 0.0

nb_drives = qubitring_ndrives(n)
hs = np.stack([_.full() for _ in qubitring_all_ops(n)])

_epsilons = [0.1]*n
_etas = [0.01]*n
_deltas = [2.]*qubitring_ndrives(n)
jitter = 0.05
s = np.random.RandomState(seed=36)
_epsilons *= s.normal(loc=1., scale=jitter, size=n)
_etas     *= s.normal(loc=1., scale=jitter, size=n)
_deltas   *= s.normal(loc=1., scale=jitter, size=qubitring_ndrives(n))

H0 = qubitring_H0(n, _epsilons, _etas).full()
Hdrives = np.stack([_.full() for _ in qubitring_Hdrives(n, _deltas)])

val_seed = 1
train_seed = 42
validation_data = next(qubitring_datagen(H0, Hdrives, n=n, batch_size=512, Delta=1., seed=val_seed, average_over=float('inf')))

gen_train = qubitring_datagen(H0, Hdrives, n=n, batch_size=512, Delta=1.,
                              seed=train_seed, average_over=average_over, step_reset=batches_worth_of_data,
                              spam=SPAM)
data_est_std = next(qubitring_datagen(H0, Hdrives, n=n, batch_size=512, Delta=1.,
                                      seed=train_seed, average_over=average_over,
                                      spam=SPAM))
# data_est_std is a "copy" of the first batch of gen_train

# Parameter estimation

- start with Low Parameterization where you specify the permitted Hamiltonian components;
- then move to High Parameterization (with linear drives), seeding it with the previous results;
- if necessary permit non-linear drives [not a default implementation];
- either one of the previous three models can be embedded in a time-dependent unitary model:
    - the unitary time-dependent model permits advanced integration methods (not just Euler), hence you need fewer timesteps (you have better scaling);
- you cal also move to non-unitary time-dependent models:
    - you have to specify the Lindblad operators;
    - only naive integration (Euler) is available, so you will need more timesteps.
- either one of the time-dependent models can be used in reverse in order to train a control pulse for a target state preparation or a target unitary [not a default implementation].

# Low Parameterization

In [8]:
model = Sequential()
model.add(StateProbabilitiesPaulied(nqubits=n, ndrives=nb_drives, paulies=hs,
                                    l1_lambda=2e-3))
model.compile(loss='mse', optimizer=Adam(lr=1e-3), metrics=['mse', 'mae', 'binary_crossentropy'])

In [9]:
hists = []

In [10]:
guess_size = 10
guess_distance = 0.1
gen_guess = qubitring_datagen(H0, Hdrives, n=n, batch_size=512, Delta=1.,
                              seed=train_seed, average_over=average_over, step_reset=1)
noise = 0.30
_epsilons_false = np.random.normal(loc=1., scale=noise, size=n)*_epsilons
_etas_false     = np.random.normal(loc=1., scale=noise, size=n)*_etas
_deltas_false   = np.random.normal(loc=1., scale=noise, size=qubitring_ndrives(n))*_deltas
guess_w_b = lambda : qubitring_perfect_pauli_weights(n, _epsilons_false, _etas_false, _deltas_false, noise=guess_distance)

starts = []
best = float('inf')
for i in range(guess_size):
    print('\rbest:',best, end='', flush=True)
    w, b = guess_w_b()
    model.set_weights([w, b])
    start = model.fit_generator(gen_guess, verbose=0, steps_per_epoch=1, epochs=1, callbacks=[]).history
    starts.append(start)
    if start['mean_squared_error'][-1] < best:
        best = start['mean_squared_error'][-1]
        best_start = start
        best_ws = [w, b]
print()
model.set_weights(best_ws)
hists.append(best_start)

best: 0.013242281338272318


In [11]:
callbacks = ([
    MacKayRegularization(data_est_std),
    CalcLogMSE(),
    ReduceLROnPlateau(monitor='log_mse', epsilon=0.01, min_lr=1e-7, patience=6),
    ModelCheckpoint(model_checkpoint_filepath, monitor='val_loss', save_best_only=True, save_weights_only=True),
    EarlyStopping(monitor='val_loss', patience=10),
    TerminateOnNaN(),
])
hists.append(model.fit_generator(gen_train, verbose=2, steps_per_epoch=steps_per_epoch, epochs=max_epochs,
                                 callbacks=callbacks,
                                 max_queue_size=500, workers=1, validation_data=validation_data).history)
jhists = join_hists(hists)

Epoch 1/50
 - 0s - loss: 0.0139 - mean_squared_error: 0.0132 - mean_absolute_error: 0.0727 - binary_crossentropy: 0.3530 - val_loss: 0.0147 - val_mean_squared_error: 0.0140 - val_mean_absolute_error: 0.0738 - val_binary_crossentropy: 0.3502
Epoch 2/50
 - 0s - loss: 0.0138 - mean_squared_error: 0.0131 - mean_absolute_error: 0.0724 - binary_crossentropy: 0.3527 - val_loss: 0.0146 - val_mean_squared_error: 0.0139 - val_mean_absolute_error: 0.0737 - val_binary_crossentropy: 0.3500
Epoch 3/50
 - 0s - loss: 0.0136 - mean_squared_error: 0.0130 - mean_absolute_error: 0.0721 - binary_crossentropy: 0.3524 - val_loss: 0.0145 - val_mean_squared_error: 0.0139 - val_mean_absolute_error: 0.0736 - val_binary_crossentropy: 0.3498
Epoch 4/50
 - 0s - loss: 0.0135 - mean_squared_error: 0.0128 - mean_absolute_error: 0.0718 - binary_crossentropy: 0.3521 - val_loss: 0.0145 - val_mean_squared_error: 0.0138 - val_mean_absolute_error: 0.0735 - val_binary_crossentropy: 0.3496
Epoch 5/50
 - 0s - loss: 0.0134 - me

# High Parameterization

In [12]:
w, b = model.get_weights()
w, b = pauli_to_arb_weights(w,b,hs)

In [13]:
kl1 = K.get_value(model.layers[0].regularizer_k.l1)
bl1 = K.get_value(model.layers[0].regularizer_b.l1)
lr = K.get_value(model.optimizer.lr)

In [14]:
hists1 = []

In [15]:
model1 = Sequential()
model1.add(StateProbabilities(nqubits=n, ndrives=nb_drives,
                              l1_lambda=2e-3))
model1.compile(loss='mse', optimizer=Adam(lr=1e-4), metrics=['mse', 'mae', 'binary_crossentropy'])
K.set_value(model1.layers[0].regularizer_k.l1, kl1)
K.set_value(model1.layers[0].regularizer_b.l1, bl1)
#K.set_value(model1.optimizer.lr, lr)
model1.set_weights((w,b))

In [16]:
callbacks = ([
    MacKayRegularization(data_est_std),
    CalcLogMSE(),
    ReduceLROnPlateau(monitor='log_mse', epsilon=0.01, min_lr=1e-7, patience=6),
    ModelCheckpoint(model1_checkpoint_filepath, monitor='val_loss', save_best_only=True, save_weights_only=True),
    EarlyStopping(monitor='val_loss', patience=10),
    TerminateOnNaN(),
])
hists1.append(model1.fit_generator(gen_train, verbose=2, steps_per_epoch=steps_per_epoch, epochs=max_epochs,
                                   callbacks=callbacks,
                                   max_queue_size=500, workers=1, validation_data=validation_data).history)
jhists1 = join_hists(hists1)

Epoch 1/50
 - 0s - loss: 0.0099 - mean_squared_error: 0.0089 - mean_absolute_error: 0.0621 - binary_crossentropy: 0.3352 - val_loss: 0.0132 - val_mean_squared_error: 0.0122 - val_mean_absolute_error: 0.0695 - val_binary_crossentropy: 0.3429
Epoch 2/50
 - 0s - loss: 0.0099 - mean_squared_error: 0.0089 - mean_absolute_error: 0.0620 - binary_crossentropy: 0.3350 - val_loss: 0.0132 - val_mean_squared_error: 0.0122 - val_mean_absolute_error: 0.0695 - val_binary_crossentropy: 0.3428
Epoch 3/50
 - 0s - loss: 0.0098 - mean_squared_error: 0.0088 - mean_absolute_error: 0.0619 - binary_crossentropy: 0.3349 - val_loss: 0.0132 - val_mean_squared_error: 0.0122 - val_mean_absolute_error: 0.0695 - val_binary_crossentropy: 0.3428
Epoch 4/50
 - 0s - loss: 0.0098 - mean_squared_error: 0.0088 - mean_absolute_error: 0.0618 - binary_crossentropy: 0.3348 - val_loss: 0.0132 - val_mean_squared_error: 0.0122 - val_mean_absolute_error: 0.0695 - val_binary_crossentropy: 0.3427
Epoch 5/50
 - 0s - loss: 0.0098 - me

# Time dependent

Use `StateProbabilitiesTimeDep`.

See section "Validation checks" for usage examples.

# Time dependent non-unitary

Use `StateProbabilitiesTimeDepLindblad` (or `StateProbabilitiesTimeDepLindbladRK4` for higher precision).

See section "Validation checks" for usage examples.

# Validation checks

Reimplementing everything in a few different ways to ensure that the numerics work.

### Unitary checks

In [7]:
model = Sequential()
model.add(StateProbabilitiesPaulied(nqubits=n, ndrives=nb_drives, paulies=hs,
                                    l1_lambda=2e-3))
model.compile(loss='mse', optimizer=Adam(lr=1e-3), metrics=['mse', 'mae', 'binary_crossentropy'])
w=model.get_weights()
x = next(gen_train)[0][:1]

np.set_printoptions(precision=3, linewidth=90)

def repeat(x, timesteps):
    x_rep = np.vstack([np.squeeze(x)]*timesteps)
    x_rep.shape = 1, *x_rep.shape
    return x_rep

def init(size):
    s = np.zeros(size)
    s[0] = 1
    return s

def low_to_H(ws, d, hs):
    w, b = ws
    return np.tensordot(hs, w.T.dot(d)+b, [[0],[0]])

def high_to_H(ws, d):
    w, b = ws
    size = int(np.sqrt(b.shape[0])+0.01)
    M = np.reshape(w.T.dot(d)+b, (size,size))
    return (M+M.T)+1j*(M-M.T)

def prop_model_low(nqubits, ndrives, w, x, hs):
    stateprob = StateProbabilitiesPaulied(nqubits=nqubits, ndrives=ndrives,
                                          paulies=hs,
                                          l1_lambda=2e-3)
    model = Sequential()
    model.add(stateprob)
    model.compile(loss='mse', optimizer=Adam(lr=1e-4),
                  metrics=['mse', 'mae', 'binary_crossentropy'])
    model.set_weights(w)
    return model.predict(x)[0]

def prop_model_high(nqubits, ndrives, w, x, hs):
    stateprob = StateProbabilities(nqubits=nqubits, ndrives=ndrives,
                                   l1_lambda=2e-3)
    model = Sequential()
    model.add(stateprob)
    model.compile(loss='mse', optimizer=Adam(lr=1e-4),
                  metrics=['mse', 'mae', 'binary_crossentropy'])
    model.set_weights(pauli_to_arb_weights(*w,hs))
    return model.predict(x)[0]

def prop_model_highT(nqubits, ndrives, w, x, hs, timesteps=100, taylorord=3, normalize=False):
    stateprob = StateProbabilities(nqubits=nqubits, ndrives=ndrives,
                                   l1_lambda=2e-3)
    stateprobT = StateProbabilitiesTimeDep(timesteps=timesteps, baseham=stateprob,
                                           normalize=normalize, taylorord=taylorord)
    model = Sequential()
    model.add(stateprobT)
    model.compile(loss='mse', optimizer=Adam(lr=1e-4),
                  metrics=['mse', 'mae', 'binary_crossentropy'])
    model.set_weights(pauli_to_arb_weights(*w,hs))
    return model.predict(repeat(x, timesteps))[0]

def prop_model_highTLindblad(nqubits, ndrives, w, x, hs, timesteps=100, normalize=False, hermify=False):
    stateprob = StateProbabilities(nqubits=nqubits, ndrives=ndrives,
                                   l1_lambda=2e-3)
    stateprobT = StateProbabilitiesTimeDepLindblad(
        timesteps=timesteps, baseham=stateprob,
        normalize=normalize, hermify=hermify)
    model = Sequential()
    model.add(stateprobT)
    model.compile(loss='mse', optimizer=Adam(lr=1e-4),
                  metrics=['mse', 'mae', 'binary_crossentropy'])
    model.set_weights(pauli_to_arb_weights(*w,hs))
    return model.predict(repeat(x, timesteps))[0]

def prop_model_highTLindbladRK4(nqubits, ndrives, w, x, hs, timesteps=100, normalize=False, hermify=False):
    stateprob = StateProbabilities(nqubits=nqubits, ndrives=ndrives,
                                   l1_lambda=2e-3)
    stateprobT = StateProbabilitiesTimeDepLindbladRK4(
        timesteps=timesteps, baseham=stateprob,
        normalize=normalize, hermify=hermify)
    model = Sequential()
    model.add(stateprobT)
    model.compile(loss='mse', optimizer=Adam(lr=1e-4),
                  metrics=['mse', 'mae', 'binary_crossentropy'])
    model.set_weights(pauli_to_arb_weights(*w,hs))
    return model.predict(repeat(x, timesteps))[0]

def prop_taylor(h, state, order=3):
    '''Taylor exp: sum (-ih)**j/j! for j in 0..order'''
    out = state
    for i in range(order,0,-1):
        out = state-1j/i*np.dot(h,out)
    return out

def prop_euler(h, state, N=10):
    '''Euler method: (1-ih/N)**N'''
    hNp1 = np.eye(len(h))+1j*h/N
    for _ in range(N):
        state = hNp1.dot(state)
    return state

def prop_eulertaylor(h, state, order=3, N=10):
    '''Euler+Taylor: (sum (-ih/N)**j/j! for j in 0..order)**N'''
    for _ in range(N):
        state = prop_taylor(h/N, state, order=order)
    return state

def prop_eulereig(h, state, N=10):
    '''Euler+Eig: exp(-ih/N)**N'''
    for _ in range(N):
        state = prop_eig(h/N, state)
    return state

def prop_pade(h, state):
    '''Pade approximation of exp(-ih)'''
    return scipy.linalg.expm(-1j*h).dot(state)

def prop_eig(h, state):
    '''Eigendecomposition for exp'''
    e,v = np.linalg.eigh(h)
    pre = np.conj(v.T)
    post = v
    return post.dot(np.exp(-1j*e)*pre.dot(state))

amp_to_prob = lambda _:np.abs(_)**2
f_args = [
    (prop_pade, {}),
    (prop_eig, {}),
    (prop_euler, dict(N=100)),
    (prop_taylor, dict(order=5)),
    (prop_taylor, dict(order=15)),
    (prop_taylor, dict(order=25)),
    (prop_eulertaylor, dict(N=100, order=3)),
    (prop_eulereig, dict(N=100)),
]
f_args_models = [
    (prop_model_low, {}),
    (prop_model_high, {}),
    (prop_model_highT, dict(timesteps=100, taylorord=1, normalize=False)),
    (prop_model_highT, dict(timesteps=100, taylorord=3, normalize=False)),
    (prop_model_highT, dict(timesteps=100, taylorord=10, normalize=False)),
    (prop_model_highT, dict(timesteps=100, taylorord='eig', normalize=False)),
    (prop_model_highT, dict(timesteps=1000, taylorord=1, normalize=False)),
    (prop_model_highT, dict(timesteps=1000, taylorord=3, normalize=False)),
    (prop_model_highT, dict(timesteps=1000, taylorord=10, normalize=False)),
    (prop_model_highT, dict(timesteps=1000, taylorord='eig', normalize=False)),
    (prop_model_highT, dict(timesteps=100, taylorord=3, normalize=True)),
    (prop_model_highT, dict(timesteps=1000, taylorord=3, normalize=True)),
    (prop_model_highTLindblad, dict(timesteps=1000, hermify=False, normalize=False)),
    (prop_model_highTLindblad, dict(timesteps=1000, hermify=False, normalize=True)),
    (prop_model_highTLindblad, dict(timesteps=1000, hermify=True, normalize=False)),
    (prop_model_highTLindblad, dict(timesteps=1000, hermify=True, normalize=True)),
    (prop_model_highTLindblad, dict(timesteps=10000, hermify=False, normalize=False)),
    (prop_model_highTLindbladRK4, dict(timesteps=10, hermify=False, normalize=False)),
    (prop_model_highTLindbladRK4, dict(timesteps=10, hermify=False, normalize=True)),
    (prop_model_highTLindbladRK4, dict(timesteps=10, hermify=True, normalize=False)),
    (prop_model_highTLindbladRK4, dict(timesteps=10, hermify=True, normalize=True)),
    (prop_model_highTLindbladRK4, dict(timesteps=100, hermify=False, normalize=False)),
]
i = init(2**n)
low = low_to_H(w, x[0], hs)
high = high_to_H(pauli_to_arb_weights(*w,hs), x[0])
print('#')
print('# Testing different parameterizations')
print('#')
print('difference in low and high H: ', np.sum(np.abs(high-low)))
print('#')
print('# Testing NUMPY models')
print('#')
pade_amp = prop_pade(low, i)
pade_prob = np.abs(pade_amp)**2
for f,args in f_args:
    print(f.__name__,args)
    amp = f(low, i, **args)
    prob = amp_to_prob(amp)
    print('    %s...] '%str(prob[:3])[:-2], end='')
    print('Sum-1: %.3e'%(np.sum(prob)-1), 'Pade diff: %.3e'%np.sum(np.abs(prob-pade_prob)))
print('#')
print('# Testing TENSORFLOW models')
print('#')
for f,args in f_args_models:
    print(f.__name__, args)
    prob = f(n, nb_drives, w, x, hs, **args)
    print('    %s...] '%str(prob[:3])[:-2], end='')
    print('Sum-1: %.e'%(np.sum(prob)-1), 'Pade diff: %.3e'%np.sum(np.abs(prob-pade_prob)))

#
# Testing different parameterizations
#
difference in low and high H:  6.915692425668235e-16
#
# Testing NUMPY models
#
prop_pade {}
    [0.961 0.003 0.03...] Sum-1: 4.441e-16 Pade diff: 0.000e+00
prop_eig {}
    [0.961 0.003 0.03...] Sum-1: 6.661e-16 Pade diff: 2.690e-16
prop_euler {'N': 100}
    [9.612e-01 9.041e-04 3.450e-0...] Sum-1: 4.180e-04 Pade diff: 9.727e-03
prop_taylor {'order': 5}
    [0.961 0.003 0.03...] Sum-1: 2.053e-06 Pade diff: 2.982e-06
prop_taylor {'order': 15}
    [0.961 0.003 0.03...] Sum-1: -1.110e-16 Pade diff: 4.546e-16
prop_taylor {'order': 25}
    [0.961 0.003 0.03...] Sum-1: -1.110e-16 Pade diff: 4.546e-16
prop_eulertaylor {'N': 100, 'order': 3}
    [0.961 0.003 0.03...] Sum-1: -3.746e-10 Pade diff: 4.388e-10
prop_eulereig {'N': 100}
    [0.961 0.003 0.03...] Sum-1: -1.954e-14 Pade diff: 2.004e-14
#
# Testing TENSORFLOW models
#
prop_model_low {}
    [0.961 0.003 0.03...] Sum-1: -2e-15 Pade diff: 1.902e-15
prop_model_high {}
    [0.961 0.003 0.03...] Sum-1

Instructions for updating:
keep_dims is deprecated, use keepdims instead


    [0.961 0.003 0.03...] Sum-1: 2e-16 Pade diff: 1.164e-10
prop_model_highT {'timesteps': 1000, 'taylorord': 3, 'normalize': True}
    [0.961 0.003 0.03...] Sum-1: 0e+00 Pade diff: 1.185e-13
prop_model_highTLindblad {'timesteps': 1000, 'hermify': False, 'normalize': False}
    [0.961 0.003 0.03...] Sum-1: 4e-16 Pade diff: 7.043e-05
prop_model_highTLindblad {'timesteps': 1000, 'hermify': False, 'normalize': True}
    [0.961 0.003 0.03...] Sum-1: 0e+00 Pade diff: 7.043e-05
prop_model_highTLindblad {'timesteps': 1000, 'hermify': True, 'normalize': False}
    [0.961 0.003 0.03...] Sum-1: 4e-16 Pade diff: 7.043e-05
prop_model_highTLindblad {'timesteps': 1000, 'hermify': True, 'normalize': True}
    [0.961 0.003 0.03...] Sum-1: 0e+00 Pade diff: 7.043e-05
prop_model_highTLindblad {'timesteps': 10000, 'hermify': False, 'normalize': False}
    [0.961 0.003 0.03...] Sum-1: -4e-15 Pade diff: 7.042e-06
prop_model_highTLindbladRK4 {'timesteps': 10, 'hermify': False, 'normalize': False}
    [0.961 

### Non-unitary checks

In [8]:
import qutip

In [9]:
h = low
i = qutip.identity(2).full()
s = qutip.sigmam().full()
k = lambda a,b,c: np.kron(a,np.kron(b,c))
ls = k(s,i,i)*0.1,k(i,s,i)*0.2,k(i,i,s)*0.3 
state = np.array([1]+[0]*(h.shape[0]-1))

def propNU_qutip_mesolve(h,ls,state):
    n = h.shape[0]
    h = qutip.Qobj(h)
    ls = [qutip.Qobj(_) for _ in ls]
    r = qutip.mesolve(h,qutip.Qobj(state),np.linspace(0,1,1000),c_ops=ls)
    return np.abs(r.states[-1].full().diagonal()) 

def propNU_model_highTLindblad(nqubits, ndrives, w, x, hs, ls, timesteps=100, normalize=False, hermify=False):
    stateprob = StateProbabilities(nqubits=nqubits, ndrives=ndrives,
                                   l1_lambda=2e-3)
    stateprobT = StateProbabilitiesTimeDepLindblad(
        timesteps=timesteps, baseham=stateprob,
        lindblads=ls,
        normalize=normalize, hermify=hermify)
    model = Sequential()
    model.add(stateprobT)
    model.compile(loss='mse', optimizer=Adam(lr=1e-4),
                  metrics=['mse', 'mae', 'binary_crossentropy'])
    model.set_weights([*pauli_to_arb_weights(*w,hs),np.array([1,1,1])])
    return model.predict(repeat(x, timesteps))[0]

def propNU_model_highTLindbladRK4(nqubits, ndrives, w, x, hs, ls, timesteps=100, normalize=False, hermify=False):
    stateprob = StateProbabilities(nqubits=nqubits, ndrives=ndrives,
                                   l1_lambda=2e-3)
    stateprobT = StateProbabilitiesTimeDepLindbladRK4(
        timesteps=timesteps, baseham=stateprob,
        lindblads=ls,
        normalize=normalize, hermify=hermify)
    model = Sequential()
    model.add(stateprobT)
    model.compile(loss='mse', optimizer=Adam(lr=1e-4),
                  metrics=['mse', 'mae', 'binary_crossentropy'])
    model.set_weights([*pauli_to_arb_weights(*w,hs),np.array([1,1,1])])
    return model.predict(repeat(x, timesteps))[0]

In [10]:
qt = propNU_qutip_mesolve(h,ls,state)
sum(qt)

0.9999999999999998

In [11]:
lb = propNU_model_highTLindblad(n, nb_drives, w, x, hs, np.array(ls), timesteps=100, hermify=False, normalize=False)
sum(lb)

1.0000000000000007

In [12]:
lbrk = propNU_model_highTLindbladRK4(n, nb_drives, w, x, hs, np.array(ls), timesteps=100, hermify=False, normalize=False)
sum(lbrk)

0.9999999999999997

In [13]:
np.sum(np.abs(qt-lb))

0.0005491377049087166

In [14]:
np.sum(np.abs(qt-lbrk))

8.081380278575183e-08

In [15]:
lbrk = propNU_model_highTLindbladRK4(n, nb_drives, w, x, hs, np.array(ls), timesteps=50, hermify=False, normalize=False)
np.sum(np.abs(qt-lbrk))

8.084694428083395e-08

In [16]:
lbrk = propNU_model_highTLindbladRK4(n, nb_drives, w, x, hs, np.array(ls), timesteps=10, hermify=False, normalize=False)
np.sum(np.abs(qt-lbrk))

1.0544385381283444e-07

In [20]:
lbrk = propNU_model_highTLindbladRK4(n, nb_drives, w, x, hs, np.array(ls), timesteps=5, hermify=False, normalize=False)
np.sum(np.abs(qt-lbrk))

5.931738377163836e-07